# 15장 RNN과 CNN 활용: 순차데이터 처리

#### 감사의 글

자료를 공개한 저자 오렐리앙 제롱과 강의자료를 지원한 한빛아카데미에게 진심어린 감사를 전합니다.

## 소개

* 시계열 데이터 분석 활용
    * 주식가격 예측
    * 차의 이동경로 예측

* 임의의 길이를 갖는 순차데이터(시퀀스) 처리 가능
    * 문장, 문서, 오디오 샘플 등

* 자동번역, 받아쓰기 등 자연어처리(NLP) 분야에 매우 유용함.

## 주요 내용

* 순환신경망(RNN) 기본개념 및 역전파 네트워크 훈련 방법 소개

* 시계열 예측 수행

* RNN의 주요 난제 설명
    * 불안정한 그레이디언트 다루기: 순환 드롭아웃 및 순환층 정규화        
    * 매우 제한적인 단기기억 문제 다루기: LSTM과 GRU 셀 활용

## 순환뉴런과 순환층

* 지금까지 살펴본 신경망 모델은 아래층에서 위층으로 학습결과를 전달하는 __전방향전달__(feedforward) 방식 사용

* 순환신경망(recurrent neural network, RNN)도 비슷하게 작동

* 차이점: 후방향으로 전달하여 학습결과를 순환시키는 기능 활용

### 순환뉴런과 타입스텝

* 순환뉴런: 입력을 받아 출력을 만든 후 자신에게도 출력결과를 전달

<div align="center"><img src="images/ch15/homl15-01a.png" width="450"/></div>

### 순환층

* 순환뉴런으로 구성된 층

<div align="center"><img src="images/ch15/homl15-02a.png" width="500"/></div>

#### 하나의 샘플에 대한 순환층의 출력

* 가중치 행렬
    * $\mathbf W_x$: 현재 타임스텝의 입력에 대한 연결가중치 행렬
    * $\mathbf W_y$: 이전 타입스텝의 출력에 대한 연결가중치 행렬
    * 타임스텝에 의존하지 않음.

<div align="center"><img src="images/ch15/homl15-05.png" width="200"/></div>

#### 배치에 대한 순환층의 출력

<div align="center"><img src="images/ch15/homl15-06.png" width="300"/></div>

### (메모리) 셀

* 메모리 셀: 타임스텝에 걸쳐서 특정 상태를 보존하는 신경망의 구성요소

* 예제: 하나의 순환뉴런 또는 순환뉴런층
    * 10 스텝 정도의 짧은 패턴을 학습하는 기본적인 셀

* 상태 $\mathbf h_{(t)}$: 타임스텝 $t$ 에서의 셀의 상태

    $$
    \mathbf h_{(t)} = f(\mathbf h_{(t-1)}, \mathbf x_{(t)})
    $$

<div align="center"><img src="images/ch15/homl15-03.png" width="350"/></div>

### 입력/출력 순차데이터

* 목적에 따라 입력/출력 데이터의 형식을 벡터 또는 순차데이터(시퀀스)로 지정할 수 있음.

#### seq-to-seq

* 입력: 순차데이터
* 출력: 순차데이터

* 예제: 주식가격 시계열 데이터 활용에 유용
    * 최근 N일 동안의 주식가격을 입력하면 각 입력값보다 하루 이후의 가격을 출력

<div align="center"><img src="images/ch15/homl15-04a.png" width="300"/></div>

#### seq-to-vector

* 입력: 순차데이터
* 출력: 벡터

* 예제: 영화리뷰
    * 영화리뷰에 있는 연속된 단어를 주입하면 감성점수 출력(-1에서 1사이)

<div align="center"><img src="images/ch15/homl15-04b.png" width="250"/></div>

#### vector-to-seq

* 입력: 벡터
* 출력: 순차데이터

* 예제: 이미지를 입력하면 이미지 캡션 출력

<div align="center"><img src="images/ch15/homl15-04c.png" width="250"/></div>

#### encoder-decoder

* 인코더와 디코더의 조합을 사용하는 이중단계모델
    * 인코더: seq-to-vector 신경망
    * 디코더: vector-to-seq 신경망

* 예제: 언어 번역
    * seq-to-seq 모델보다 성능 좋음.
    * 이유: "한국말은 끝까지 들어야 한다" 와 동일한 이유이며,
        하나의 입력문장이 끝날 때까지 기다린 후 그 결과를 번역하는 과정에 적합함. (16장 참조)

<div align="center"><img src="images/ch15/homl15-04d.png" width="330"/></div>

## RNN 훈련

* BPTT(backpropagation through time) 전략: 타임스텝을 통과하는 역전파 전략


* 가중치 $\textbf{W}$와 편향 $\textbf{b}$가 타임스텝에 의존하지 않기에 역전파가 모든 관련된
    타임스텝에 대해 일관되게 작동함.


<div align="center"><img src="images/ch15/homl15-07b.png" width="400"/></div>

## 시계열 예측

### 단변량 시계열

* 타임스텝마다 하나의 값을 가지는 순차데이터

* 예제
    * 웹사이트의 시간당 접속자 수
    * 도시의 날짜별 온도

### 다변량 시계열

* 타임스텝마다 여러 개의 값을 가지는 순차데이터

* 예제
    * 회사의 수입, 부채 등 여러 지표를 사용한 기업의 분기별 재정안정성

### 활용법 두 가지

* 예측: 미래의 값 추정하기

* 결측치 대체: 과거 데이터에서 누락된 값 추정하기

### 예제

* 아래 그림: 3개의 단변량 시계열
    * 타임스텝: 50개
    * 목표: `X` 로 표시된 값 예측하기

<div align="center"><img src="images/ch15/homl15-08.png" width="550"/></div>

* 위와 같은 시계열 데이터 10,000개를 훈련 데이터로 생성하여 순환신경망(RNN) 학습법을 살펴보고자 함.

### 훈련 시계열 데이터 준비

* 아래 함수를 이용하여 시계열 데이터 생성

* 아래 함수의 반환값의 모양: `(배치 크기, 타임스텝, 1)`
    * 마지막 값이 1인 이유: 단변량 시계열 데이터를 생성하기 때문

```python
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   파도 곡선 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 파도 속선 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)
```

* 길이가 50인 1만 개의 시계열 데이터를 생성하여 각각 7천 개 2천 개, 천 개 크기의 훈련, 검증, 테스트 세트 생성
    * 예를 들어 `X_train`과 `t_train`의 모양은 각각 `(7000, 50, 1)`과 `(7000, 1)` 임.

```python
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]
```

### RNN 성능 비교

* RNN의 성능을 타 방식과 비교하기 위해 두 가지 방식의 성능을 먼저 계산해 보고자 함.
    * 순진한 예측
    * 선형회귀 예측

#### 순진한 예측

* 시계열의 마지막 값을 예측값으로 사용

```python
y_pred = X_valid[:, -1]
```

* 평균제곱오차(MSE)가 0.02 정도로 매우 우수한 것처럼 보임

```python
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))
```

#### 선형회귀 예측

* 완전연결 네트워크를 사용하여 선형회귀 예측 실행
    * 입력값을 1차원 배열로 변환해서 사용함.

```python
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
    ])

model.compile(loss="mse", optimizer="adam")
```

* 검증 세트에 대한 평균제곱오차(MSE)가 0.0041 정도로 훨씬 우수함

#### 가장 단순한 RNN 모델 예측 성능

* 한 개의 순환뉴런을 갖는 `SimpleRNN` 순환층 한 개로 구성된 RNN 모델
    * 입력값의 크기, 즉, 타임스텝의 크기가 임의이어도 되기에 입력값의 모양의 첫째 항목이 `None`으로 지정됨.

```python
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="mse", optimizer=optimizer)
```

* `SimpleRNN` 층의 특징
    * 활성화 함수: `tanh`
    * 반환값: 최종 출력만 리턴하는 게 기본. 모든 타임스텝에서 출력하도록 하려면 `return_sequences=True`로 설정.
    * 모든 시계열 샘플에 데이터를 동시에 처리.
    

* 검증 세트에 대한 평균제곱오차(MSE)가 0.0109 정도로 선형회귀 예측보다 좋지 않음.

* 하지만 선형회귀 모델은 51개의 파라미터를 사용하는 반면에 위 RNN 모델은 단 세 개의 파라미터만 사용함.

* 선형회귀 모델의 파라미터
    * 50 개의 입력값에 대한 가중치 + 편향

* 위 RNN 모델의 파라미터
    * 순환층에 사용된 순환뉴런의 수를 지정하는 파라미터 + 입력값에 대한 1개의 가중치 + 편향

#### 심층 RNN 모델 예측 성능

* 셀을 여러 층으로 쌓은 RNN

<div align="center"><img src="images/ch15/homl15-09.png" width="400"/></div>

* 마지막 층을 제외한 순환층은 `return_sequences=True`을 반드시 사용해야 함.

```python
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")
```

* 검증 세트에 대한 평균제곱오차(MSE)가 0.0029 정도로 선형회귀 예측보다 두 배 정도 좋아졌음.

<div align="center"><img src="images/ch15/homl15-10.png" width="400"/></div>

* 하지만 위 모델은 아래 두 가지 관점에서 개선될 수 있음.
    * 첫째, 단변량 시계열 데이터를 사용하기에 마지막 순환층이 하나의 순환뉴런으로 구성되어야 함.
        이는 전달되는 (은닉)상태가 하나의 숫자에 불과함을 의미하며 많은 정보를 담고 있지 않음.
        따라서 분명이 이전 층의 (은닉) 정보가 많이 활용될 것임.
    * 둘째, `SimpleRNN` 층의 활성화 함수가 `tanh`로 고정이어서 다른 활성화 함수를 사용 불가.

* 위 문제를 해결하기 위해 마지막 출력 층을 밀집층으로 바꾸고, 활성화 함수도 원하는 대로 설정할 수 있음.
    * 성능은 비슷하고 훈련속도는 빨라짐.

```python
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
```

<div align="center"><img src="images/ch15/homl15-11.png" width="400"/></div>